In [ ]:
!pip3 install tfrecord
!pip3 install pytorch_lightning

In [ ]:
import torch
import torch.nn as nn
import torchvision

import numpy as np

from BReGNeXt import BReGNeXt as Classif
import cv2

import os
from itertools import groupby

import tfrecord
import pytorch_lightning

from BReGNeXt import BReGNeXt
from utils import ShuffleDataset

import os

cuda = torch.device('cuda:0')

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/MyDrive/train_set.zip -d train_set

In [ ]:
import shutil as shtl

files = os.listdir('val_set/images')
for file in files:
    os.rename('val_set/images/' + file, 'val_set/images/' + file.split('.')[0])

In [ ]:
sorted(os.listdir('train_set/images'))

In [ ]:
tr_files = torch.tensor(np.array(sorted(os.listdir('train_set/images')), dtype=int))
tr_labels = torch.tensor(np.load('train_set/labels/labels.npy'))

tr_dset = torch.utils.data.TensorDataset(tr_files, tr_labels)
tr_dset

In [ ]:
val_files = torch.tensor(np.array(sorted(os.listdir('val_set/images')), dtype=int))
val_labels = torch.tensor(np.load('val_set/labels/labels.npy'))

val_dset = torch.utils.data.TensorDataset(val_files, val_labels)
val_dset

In [ ]:
def collate_tr(bdata):
    inp = torch.zeros(len(bdata), 3, 64, 64)
    labels = torch.zeros(len(bdata), dtype=torch.long)

    for i, (file, label) in enumerate(bdata):

        im = cv2.imread('train_set/images/' + str(file.item()))
        inp[i] = decode_and_preprocess_image(im)
        labels[i] = label
    return inp.to(cuda), labels.to(cuda)

tr_dloader = torch.utils.data.DataLoader(tr_dset, batch_size=128, collate_fn=collate_tr, shuffle=True)

In [ ]:
def collate_val(bdata):
    inp = torch.zeros(len(bdata), 3, 64, 64)
    labels = torch.zeros(len(bdata), dtype=torch.long)

    for i, (file, label) in enumerate(bdata):

        im = cv2.imread('val_set/images/' + str(file.item()))
        inp[i] = decode_and_preprocess_image(im)
        labels[i] = label
    return inp.to(cuda), labels.to(cuda)

val_dloader = torch.utils.data.DataLoader(val_dset, batch_size=64, collate_fn=collate_val)

In [ ]:
for i, (im, labels) in enumerate(val_dloader):
    print(i)

In [ ]:
train_data_path = 'training_FER2013_sample.tfrecords'
val_data_path = 'validation_FER2013_sample.tfrecords'

train_dataset = ShuffleDataset(tfrecord.torch.dataset.TFRecordDataset(
    data_path=train_data_path,
    index_path=None,
    description={'image_raw': 'byte', 'label': 'int'},
    transform=decode_and_preprocess_image,
), 1024)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, num_workers=2)

valid_dataset = tfrecord.torch.dataset.TFRecordDataset(
    data_path=val_data_path,
    index_path=None,
    description={'image_raw': 'byte', 'label': 'int'},
    transform=decode_and_preprocess_image,
)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, num_workers=2)

In [ ]:
classif = Classif(n_classes=8).to(cuda)

In [ ]:
import numpy as np

train_dataset = tfrecord.torch.dataset.TFRecordDataset(
    data_path=train_data_path,
    index_path=None,
    description={'image_raw': 'byte', 'label': 'int'},
    transform=decode_and_preprocess_image,
)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64)

tr_ims = torch.tensor([])
tr_labels = torch.tensor([])
for i, batch in enumerate(valid_dataloader):
    tr_ims = torch.cat([tr_ims, batch], dim=0)
    tr_labels = torch.cat([tr_labels, batch['label'].reshape(-1)], dim=0)
    print(i)

In [ ]:
from itertools import groupby 
import cv2

class PackLoader():
    def __init__(self, psize, path):
        
        self.path = path
        lfiles = np.array(os.listdir(path + '/annotations'))
        idx = [x.split('_')[1] == 'exp.npy' for x in lfiles]
        lfiles = lfiles[idx]
        nums = [x.split('_')[0] for x in lfiles]
        files = [path + '/images/' + num + '.jpg' for num in nums]
        lfiles = [path + '/annotations/' + x for x in lfiles]
            
        self.files = np.array(files)
        self.labels = np.array([int(np.load(labfile).item()) for labfile in lfiles])
        self.idx = np.random.permutation(len(files))
        self.psize = psize

        return

    def pack(self, i):
        idx = self.idx[i * self.psize:(i + 1) * self.psize]
    
        files = self.files[idx]
        labels = self.labels[idx]
        pack = []
        for j, file in enumerate(files):
            im = cv2.imread(file)
            pack.append((im, labels[j]))
            
        return pack

In [ ]:
from itertools import groupby 
import cv2

class PackLoader():
    def __init__(self, psize, path):
        
        self.path = path
        files = sorted([path + '/images/'+ imfile for imfile in os.listdir(path + '/images')])
            
        self.files = np.array(files)
        self.labels = np.load(path + '/labels/labels.npy')
        self.idx = np.random.permutation(len(files))
        self.psize = psize

        return

    def pack(self, i):
        idx = self.idx[i * self.psize:(i + 1) * self.psize]
    
        files = self.files[idx]
        labels = self.labels[idx]
        pack = []
        for j, file in enumerate(files):
            im = cv2.imread(file)
            pack.append((im, labels[j]))
            
        return pack

In [ ]:
_image_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ColorJitter(brightness=(0, 0.05), contrast=(0.7,1.3), saturation=(0.6, 1.6), hue=0.08),
    torchvision.transforms.RandomResizedCrop((64,64)),
    torchvision.transforms.ToTensor(),
])


def decode_and_preprocess_image(features):
    # Decode the image
    features['image_raw'] = features['image_raw'].reshape(64, 64, 3)
    features['image_raw'] = _image_transform(features['image_raw'])
    features['image_raw'] = features['image_raw'] - torch.FloatTensor([0.5727663, 0.44812188, 0.39362228]).unsqueeze(-1).unsqueeze(-1)
    features['label'] = torch.LongTensor(features['label'])
    return features

In [ ]:
_image_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize((64, 64)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ColorJitter(brightness=(0, 0.05), contrast=(0.7,1.3), saturation=(0.6, 1.6), hue=0.08),
    torchvision.transforms.RandomResizedCrop((64,64)),
    torchvision.transforms.ToTensor(),
])


def decode_and_preprocess_image(features):
    # Decode the image
    # features = features.reshape(64, 64, 3)
    features = _image_transform(features)
    features = features - torch.FloatTensor([0.5727663, 0.44812188, 0.39362228]).unsqueeze(-1).unsqueeze(-1)
    return features

In [ ]:
prepr = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                     std=[0.5, 0.5, 0.5])
])

In [ ]:
from torch.utils.data import DataLoader
    
def gen_batch(bdata):
    inp = torch.zeros(len(bdata), 3, 64, 64)
    labels = torch.zeros(len(bdata), dtype=torch.long)
    for i, (im, label) in enumerate(bdata):
        inp[i] = prepr(im)
        labels[i] = label
    return inp.to(cuda), labels.to(cuda)

In [ ]:
def focal_loss2(input_tensor, target_tensor, weight=None, gamma=2, reduction='mean'):
    log_prob = torch.nn.functional.log_softmax(input_tensor, dim=-1)
    probs = torch.exp(log_prob)
    # weight_tens = torch.ones((target_tensor.shape[0]), dtype=torch.float32, device=cuda) * weight
    return torch.nn.functional.nll_loss(((1 - probs) ** gamma) * log_prob,
                                        target_tensor, weight=weight, reduction=reduction)
ce_loss = nn.CrossEntropyLoss()
def cross_entropy(pred, target):
    probs = torch.nn.functional.softmax(pred, dim=-1)
    # probs = torch.exp(log_prob)
    return ce_loss(probs, target)

In [ ]:
ploader = PackLoader(1000, 'train')
ploader_val = PackLoader(200, 'test')

In [ ]:
optimizer = torch.optim.Adam(classif.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8) # !
criterion = focal_loss2
# criterion = cross_entropy

for epoch in range(100):
    # if (epoch >= 3) and (epoch % 3 == 0):
    #     scheduler.step()
        
    # loss_train = 0
    for i, batch in enumerate(train_dataloader):
        x = batch.to(cuda)
        labels = batch['label'].to(cuda).reshape(-1)
        pred = classif(x)

        optimizer.zero_grad()
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()


        # loss_train += loss.detach()

        print('Epoch {} Pack {} Loss_train {}'.format(epoch, i, loss.detach()))

    loss_val, acc = validate(classif)
    print('Epoch {} Loss_val {} Val_acc {}'.format(epoch, loss_val, acc))

    # scheduler.step() # ! м.б. убрать вообще

    torch.save(classif.state_dict(), 'bregnext_' + str(epoch))

In [ ]:
torch.save(classif.state_dict(), 'bregnext')

In [ ]:
def validate(classif):
    hits = 0
    total = 0
    loss_sum = 0
        
    for i, batch in enumerate(valid_dataloader):
        x = batch.to(cuda)
        labels = batch['label'].to(cuda).reshape(-1)
        pred = classif(x)
        loss_sum += criterion(pred, labels).detach()
        
        pr_labels = torch.argmax(pred.detach(), dim=-1)
        hits += (labels == torch.argmax(pred, dim=-1)).sum()
        total += labels.shape[0]

    return loss_sum / (i + 1), hits / total

In [ ]:
class BReGNeXtPTLDriver(pytorch_lightning.LightningModule):

    def __init__(self, use_focal_loss = False, learning_rate = 0.0001):

        super(BReGNeXtPTLDriver, self).__init__()

        self._use_focal_loss = use_focal_loss
        self.learning_rate = learning_rate
        self._model = BReGNeXt(n_classes=8)

    def training_step(self, batch, batch_idx):
        logits = self._model(batch[0])
        labels = batch[1].reshape(-1)
        loss = focal_loss2(logits, labels) if self._use_focal_loss else torch.nn.functional.cross_entropy(logits, labels)
        accuracy = (logits.argmax(dim=-1) == labels).float().mean()

        self.log('train/accuracy', accuracy, prog_bar=True)
        self.log('train/loss', loss, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        logits = self._model(batch[0])
        labels = batch[1].reshape(-1)
        loss = focal_loss2(logits, labels) if self._use_focal_loss else torch.nn.functional.cross_entropy(logits, labels)
        accuracy = (logits.argmax(dim=-1) == labels).float().mean()

        self.log('val/accuracy', accuracy, prog_bar=True)
        self.log('val/loss', loss, prog_bar=True)

        if batch_idx == 0:
            torch.save(self._model.state_dict(), 'graphs/bregnext')

        return loss

    def configure_optimizers(self):
        # optimizer = torch.optim.Adam(self._model.parameters(), lr=self.learning_rate, weight_decay=0.0001)
        optimizer = torch.optim.Adam([
            {'params': self._model._fc0.parameters(), 'lr': 1e-4},
            {'params': self._model._model.parameters(), 'lr': 1e-4, 'weight_decay': 1e-4}
        ])
        # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.80)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)
        return [optimizer], [scheduler]
        # return {'optimizer': optimizer,
        #        'lr_scheduler': {'scheduler': scheduler,
        #                         'interval': 'epoch'}}

In [ ]:
class BReGNeXtPTLDriver(pytorch_lightning.LightningModule):

    def __init__(self, use_focal_loss = False, learning_rate = 0.0001):

        super(BReGNeXtPTLDriver, self).__init__()

        self._use_focal_loss = use_focal_loss
        self.learning_rate = learning_rate
        self._model = BReGNeXt(n_classes=7)
        # self.epoch = 0
        # self.sched = torch.optim.lr_scheduler.ExponentialLR

    def training_step(self, batch, batch_idx):
        logits = self._model(batch['image_raw'])
        batch['label'] = batch['label'].reshape(-1)
        loss = focal_loss2(logits, batch['label']) if self._use_focal_loss else torch.nn.functional.cross_entropy(logits, batch['label'])
        accuracy = (logits.argmax(dim=-1) == batch['label']).float().mean()

        # if (batch_idx == 0):
        #     self.epoch += 1

        self.log('train/accuracy', accuracy, prog_bar=True)
        self.log('train/loss', loss, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        logits = self._model(batch['image_raw'])
        batch['label'] = batch['label'].reshape(-1)
        loss = focal_loss2(logits, batch['label']) if self._use_focal_loss else torch.nn.functional.cross_entropy(logits, batch['label'])
        accuracy = (logits.argmax(dim=-1) == batch['label']).float().mean()

        self.log('val/accuracy', accuracy, prog_bar=True)
        self.log('val/loss', loss, prog_bar=True)
        # self.log('lr', self.optimizer.state_dict()['param_groups'][0]['lr'], prog_bar=True)
        # self.log('epch', self.epoch, prog_bar=True)

        # if (self.epoch % 3 == 0) and (batch_idx == 0):
        #     scheduler = self.sched(self.optimizer, gamma=0.8)
        #     scheduler.step()


        return loss

    def configure_optimizers(self):
        # optimizer = torch.optim.Adam(self._model.parameters(), lr=self.learning_rate, weight_decay=0.0001)
        optimizer = torch.optim.Adam([
            {'params': self._model.base.parameters()},
            {'params': self._model._model.parameters(), 'lr': 1e-4, 'weight_decay': 1e-4}
        ], lr=1e-4)
        # self.optimizer = optimizer
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.8)
        # self.sched = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.80)
        # return [optimizer], [scheduler]
        # return {'optimizer': optimizer,
        #         'lr_scheduler': {'scheduler': scheduler,
        #                          'name': 'exp_sched'}}
        return [optimizer], [scheduler]

In [ ]:
model = BReGNeXtPTLDriver(use_focal_loss=True)

# lr_monitor = pytorch_lightning.callbacks.LearningRateMonitor(logging_interval='epoch')
trainer = pytorch_lightning.trainer.Trainer(max_epochs=100, gpus=1) # , callbacks=[lr_monitor])
# lr_monitor.on_train_epoch_start(trainer)
trainer.fit(model, tr_dloader, val_dloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type     | Params
------------------------------------
0 | _model | BReGNeXt | 3.1 M 
------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.291    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
model.configure_optimizers()
model.optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.0001
    lr: 0.0001
    weight_decay: 0.0001
)

In [ ]:
!python3 trainer.py --use_focal_loss --train_data_path training_FER2013_sample.tfrecords --val_data_path validation_FER2013_sample.tfrecords --gpus 1 --max_epoch 100

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type     | Params
------------------------------------
0 | _model | BReGNeXt | 3.1 M 
------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.291    Total estimated model params size (MB)
Validation sanity check:   0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: U